## 9-2. Quantum Error Correction

<div class="alert alert-info">
    The QEC algorithm is implemented with the `DynamicCircuit` object. This is the dynamic circuit experimental feature that has not yet been released as of quri_parts 0.17.0.
</div>

The underlying causes of error in the quantum bit itself are actually not that different from those in the classical bit.

One is the error caused by the leakage of information to the outside at a certain rate due to the interaction with the outside environment.
In particular, when reading and writing information with electromagnetic waves such as light or microwaves as a quantum bit, a path must be secured for injecting electromagnetic waves, and a certain amount of information leaks out from there.
In addition, even when experimenting with dilution refrigerators, microwaves are greatly affected by thermal noise because their energy scale is close to the ambient temperature, which is another source of noise.
On the other hand, in the case of using traps such as ions or neutral atoms, in addition to decoherence, errors that cause materials to fall out of the traps are also a problem.
When light is used as a quantum bit, the above decoherence is not a problem due to the difference in temperature scales.
However, in order to scale optical quantum bits, it is necessary to secure the path by induction or mirror reflection using a fiber, and since there are also fluctuations in the laser, the loss of photons and coherence associated with these fluctuations can be a problem.
The associated loss of photons and loss of coherence due to the passage of coherence time are effectively the lifetime of the qubit, which occurs even if nothing is done.

The second is the noise that occurs during manipulation of the device.
In the case of microwave operation of devices such as superconducting qubits, errors in the operation include noise due to fluctuations in microwave frequency and power, and bias due to unintended contributions from interaction terms.
Such noise can be divided into bias noise, which can be canceled by inversion, and stochastic noise, which is not. The result of noise can also be divided into noise that eventually moves a state to either 0 or 1 and leakage, and notise which moves a state to neither 0 nor 1. Noise such as stochastic noise and leakage is more serious in terms of error correction.

Such errors can in principle be avoided by calibrating the pulses or applying canceling pulses, but calibrating a huge number of bits is extremely difficult. The situation is similar for ions and neutral atoms.
When light is used as a qubit, it is manipulated via interactions with optical crystals, which pose different problems from those of matter.
First, the universal manipulation of optical qubits requires interactions with third-order or higher nonlinearities, and these higher-order nonlinearities are often accompanied by strong absorption.
For this reason, we often give up on nondestructive third-order nonlinear operations and perform photon measurement and feedback operations with destruction on entangled photon qubits, and achieve the desired nonlinear operation in the form of gate teleportation. Such operations involve a large overhead compared to mere nondestructive manipulation.

Also, in operations using electromagnetic waves, the qubits do not move and the electromagnetic waves are sent out with time, so the size of the system does not depend on the time to perform the computation. 
On the other hand, in the usual experiment using optical qubits, the material is fixed and placed on an optical surface plate, so the number of optical devices used increases in dependence on the computation time.
To avoid this, time multiplexing is necessary, in which an optical device is used once and then used again through a loop. Such time multiplexing of optical systems is currently the subject of intense research.

### Quantum error correction
Quantum bits are extensions of ordinary bits that allow superposition of states. In quantum error correction, the qubit is made redundant, and the goal is to obtain an effectively small error probability, as in classical error correction. In this case, the nature of qubits makes the situation different from that of many classical codes, and most of them work in the direction that makes quantum error correction difficult.

Currently proposed quantum error correction methods are, in principle, extensions of linear codes to quantum versions. For confirmation, we restate the framework of linear codes.

- code: Characterized by the generator matrix $G$ and the parity check matrix matrix $H_c$. They satisfy $GH_c = 0$.
- Before coding: $k$-bit transversal vector data $V$.
- Encoding: Using the generator matrix $G$ of $k \times n$, the encoded state $v'$ is $v'=vG$.
- Error: $n$-bit sequence $e$ is randomly added to $v'$ after encoding.
- Error checking: $n \times (n-k)$ parity check $H_c$ is used to obtain the syndrome $s = e H_c$. If $s \neq 0$, an error can be detected. If $s=0$, either no error has occurred or the error is above the limit that can be inspected.
- Error correction: Guess $k$-bit transverse vector data $v$ according to the value of $(v'+e)$. The algorithm to find $v$ from $v'+e$ is called the decoding algorithm.
- Distance: The minimum weight of $e$ such that $e H_c = 0$ is the distance $d$ of the code.

If a qubit uses only two states of $|0\rangle, |1\rangle$ and does not use superposition, it can be put on the framework of classical bits. Using this fact, the above classical error correction can be rewritten in terms of quantum information as follows.

- code: Characterized by the generator matrix $G$ and the parity check matrix $H_c$. They satisfy $GH_c = 0$.
- Before coding: there is a quantum state $|v\rangle$ of $k$-qubit of the computational basis .
- Coding: a unitary matrix $U_G$ of $2^n \times 2^n$ consisting only of CNOT and Pauli-$X$ such that $|vG\rangle = U_G (|\psi\rangle \otimes |0\rangle^{n-k})$ can be constructed from $G$.
- Error: A random bit-flip operation $E = X_1^{e_1} X_2 ^{e_2} \ldots X_n^{e_n}$ acts on the $|vG \rangle$ after encoding, resulting in $E |vG \rangle = |vG + e \rangle$.
- Error checking: Checks if the parities are consistent. This is equivalent to the following operations Let $h_i$ be the $i$ column vector of the parity check matrix $H_c$, consider the Pauli matrix $P_i = Z_1^{h_{i1}} Z_2^{h_{i2}} \ldots Z_n^{h_{in}}$ and let $M_0^{(i)} = (I+P_i)/2, M_1^{(i)} = (I-P_ i)/2$ , and consider the POVM $\{ M_0^{(i)}, M_1^{(i)} \}$ .  [POVM](https://en.wikipedia.org/wiki/POVM) is a generalization of the projective measurements; for Nielsen-Chuang, it is `2.2.6 POVM (measurements`). We measure the state $E |\psi' \rangle$ with $n-k$ POVM $\{M_0^{(i)}, M_1^{(i)}\}_i$ and obtain $n-k$ bits $s = (vG+e)H_c = eH_c$. Check whether this $s$ is zero.

- Decoding: $|vG+e'\rangle$ all measured in $Z$ basis yields $n$-bit sequence $(vG+e)$. According to the value of $(vG+e)$, we construct a unitary operation $R = X_1^{e'_1} X_2 ^{e'_2} \ldots X_n^{e'_n}$ such that $R E|vG\rangle = |vG\rangle$. The algorithm for finding $e'$ from $(vG+e)$ is called the decoding algorithm.
- Distance: The minimum weight of $E$ such that $s = 0$ is the distance $d$ of the code.

The above protocol is a framework for classical error correction with qubits, and what we are saying is equivalent.
As an example, we show a code in which one bit is increased to three bits to perform majority voting.

Generator matrix is

$$
G =\left( \begin{matrix} 1 & 1 & 1 \end{matrix} \right)
$$

Parity check matrix is 

$$
H_c = \left( \begin{matrix} 1 & 0 \\ 1 & 1 \\ 0 & 1 \end{matrix} \right)
$$

In [1]:
# set code info
repetition = 3
ancilla = 2

n_qubit = repetition + ancilla
n_cbit = ancilla

error_probability = 0.2
initial_bit = 1

In [2]:
from numpy.typing import NDArray
import numpy as np

def show_quantum_state(
    state_vector: NDArray[np.complex128], show_ancilla: bool = False,
    tol: float = 1e-10
) -> None:
    
    n_qubit = int(np.log2(len(state_vector)))
    subterm = []
    for i, v in enumerate(state_vector):
        if np.abs(v) < tol:
            continue
        ket = bin(i)[2:].zfill(n_qubit)
        if not show_ancilla:
            ket = ket[ancilla:]
        subterm.append(f"{np.round(v, 3)} * |{ket}>")

    print(" + ".join(subterm))

In [3]:
from quri_parts.qulacs.simulator import evaluate_state_to_vector
from quri_parts.core.state import ComputationalBasisState

state = ComputationalBasisState(n_qubit, bits=1)
show_quantum_state(
    evaluate_state_to_vector(state).vector
)

(1+0j) * |001>


In [4]:
from quri_parts.circuit import QuantumCircuit

encode_circuit = QuantumCircuit(n_qubit, n_cbit)
for i in range(1, repetition):
    encode_circuit.add_CNOT_gate(0, i)

encoded_state = evaluate_state_to_vector(
    state.with_gates_applied(encode_circuit)
).vector

show_quantum_state(encoded_state)

(1+0j) * |111>


Create a quantum circuit to measure the syndrome values. At the current point, all the syndrome values are 0.

In [5]:
from quri_parts.circuit import DynamicCircuit
from quri_parts.circuit.gates import X, CNOT, Measurement
from quri_parts.qulacs.simulator import run_dynamic_circuit

def get_parity_measurement_circuit():
    dynamic_circuit = DynamicCircuit(n_qubit, n_cbit)
    dynamic_circuit.add_gate(CNOT(0, 3))
    dynamic_circuit.add_gate(CNOT(1, 3))
    dynamic_circuit.add_gate(CNOT(0, 4))
    dynamic_circuit.add_gate(CNOT(2, 4))
    dynamic_circuit.add_gate(Measurement([3, 4], [0, 1]))
    return dynamic_circuit

def show_parity_of_state(state: NDArray[np.complex128]) -> tuple[int, int]:
    non_zero_component = np.where(state != 0)[0]
    parity_01 = (non_zero_component >> repetition) & 1
    parity_02 = non_zero_component >> (repetition + 1)
    print("parity(0, 1):", parity_01[0])
    print("parity(0, 2):", parity_02[0])

parity_measurement_circuit = get_parity_measurement_circuit()

parity_measured_state = run_dynamic_circuit(
    parity_measurement_circuit,
    encoded_state
)

show_quantum_state(parity_measured_state)
show_parity_of_state(parity_measured_state)

(1+0j) * |111>
parity(0, 1): 0
parity(0, 2): 0


One bit error is made on a random data qubit. At this time, a bit error is equivalent to an unintended Pauli-$X$ operation. The quantum state will be $|vG+e\rangle$.

In [6]:
from quri_parts.circuit.gates import Pauli
from quri_parts.qulacs.simulator import run_circuit

def random_X_error() -> QuantumCircuit:
    error_probability = 0.2
    
    error_array = []
    for _ in range(repetition):
        if np.random.rand() < error_probability:
            error_array.append(1)
        else:
            error_array.append(0)
    
    print(f"{error_array = }")
    return QuantumCircuit(
        n_qubit, n_cbit,
        gates=[Pauli(np.arange(n_qubit), error_array + [0, 0])]
    )

error_applied_state = run_circuit(random_X_error(), encoded_state)
show_quantum_state(error_applied_state)

error_array = [0, 0, 0]
(1+0j) * |111>


Recover the parity measurement again.

In [7]:
parity_measured_error_state = run_dynamic_circuit(parity_measurement_circuit, error_applied_state)
show_quantum_state(parity_measured_error_state)
show_parity_of_state(parity_measured_error_state)

(1+0j) * |111>
parity(0, 1): 0
parity(0, 2): 0


Decryption

In [8]:
def compute_recovery_operation(state: NDArray[np.complex128]) -> QuantumCircuit:
    non_zero_component = np.where(state != 0)[0]
    parity_bit = non_zero_component >> repetition
    assert len(set(parity_bit)) == 1

    if parity_bit == 0:
        return QuantumCircuit(n_qubit, n_cbit)
    
    elif parity_bit == 0b01:
        return QuantumCircuit(n_qubit, n_cbit, gates=[X(1)])
    
    elif parity_bit == 0b10:
        return QuantumCircuit(n_qubit, n_cbit, gates=[X(2)])
    
    elif parity_bit == 0b11:
        return QuantumCircuit(n_qubit, n_cbit, gates=[X(0)])
    
    else:
        assert False

parity_measured_state = run_dynamic_circuit(
    parity_measurement_circuit,
    error_applied_state
)
recovered_state = run_circuit(
    compute_recovery_operation(parity_measured_state),
    parity_measured_state
)
show_quantum_state(recovered_state)

(1+0j) * |111>


A series of operations can be performed even if the first quantum state is not a computational basis.

In [9]:
import numpy as np
from quri_parts.circuit import QuantumCircuit, DynamicCircuit, ConditionalCircuit
from quri_parts.circuit.gates import UnitaryMatrix, CNOT, Pauli, Measurement, X
from scipy.stats import unitary_group
from quri_parts.qulacs.simulator import run_dynamic_circuit

repetition = 3
ancilla = 2

n_qubit = repetition + ancilla
n_cbit = ancilla

def add_encode_circuit(dynamic_circuit: DynamicCircuit) -> None:
    dynamic_circuit.add_gate(CNOT(0, 1))
    dynamic_circuit.add_gate(CNOT(0, 2))

def add_noisy_bit_flip_channel(dynamic_circuit: DynamicCircuit) -> None:
    error_probability = 0.2
    
    error_array = []
    for _ in range(repetition):
        if np.random.rand() < error_probability:
            error_array.append(1)
        else:
            error_array.append(0)
    
    print(f"{error_array = }")
    gate = Pauli(np.arange(n_qubit), error_array + [0, 0])
    
    dynamic_circuit.add_gate(gate)

def add_parity_measurement_gate(dynamic_circuit: DynamicCircuit) -> None:
    dynamic_circuit.add_gate(CNOT(0, 3))
    dynamic_circuit.add_gate(CNOT(1, 3))
    dynamic_circuit.add_gate(CNOT(0, 4))
    dynamic_circuit.add_gate(CNOT(2, 4))
    dynamic_circuit.add_gate(Measurement([3, 4], [0, 1]))

def add_correction_gate(dynamic_circuit: DynamicCircuit) -> None:
    dynamic_circuit.add_conditional(
        if_block=QuantumCircuit(n_qubit, n_cbit, [X(0), X(3), X(4)]),
        else_block=None,
        classical_indices=[0, 1],
        patterns={0b11}
    )
    dynamic_circuit.add_conditional(
        if_block=QuantumCircuit(n_qubit, n_cbit, [X(1), X(3)]),
        else_block=None,
        classical_indices=[0, 1],
        patterns={0b01}
    )
    dynamic_circuit.add_conditional(
        if_block=QuantumCircuit(n_qubit, n_cbit, [X(2), X(4)]),
        else_block=None,
        classical_indices=[0, 1],
        patterns={0b10}
    )

def add_decode_gate(dynamic_circuit: DynamicCircuit) -> None:
    dynamic_circuit.add_gate(CNOT(0, 2))
    dynamic_circuit.add_gate(CNOT(0, 1))

# random unitary
random_unitary = unitary_group.rvs(2)
inital_state = random_unitary[:, 0]

dynamic_circuit = DynamicCircuit(n_qubit, n_cbit)
dynamic_circuit.add_gate(UnitaryMatrix([0], random_unitary))

inital_state = [1 if i == 1 else 0 for i in range(2**n_qubit)]
current_state = run_dynamic_circuit(dynamic_circuit, inital_state)

print("1. initial state:")
show_quantum_state(current_state)

# run QEC algorithm
print("")
print("2. encoded state:")
add_encode_circuit(dynamic_circuit)
show_quantum_state(
    run_dynamic_circuit(dynamic_circuit, inital_state)
)

print("")
print("3. noise applied state:")
add_noisy_bit_flip_channel(dynamic_circuit)
show_quantum_state(
    run_dynamic_circuit(dynamic_circuit, inital_state)
)

print("")
print("4. parity measurement state:")
add_parity_measurement_gate(dynamic_circuit)
show_quantum_state(
    run_dynamic_circuit(dynamic_circuit, inital_state),
)
show_parity_of_state(
    run_dynamic_circuit(dynamic_circuit, inital_state),
)


print("")
print("5. Apply correction gate:")
add_correction_gate(dynamic_circuit)
show_quantum_state(
    run_dynamic_circuit(dynamic_circuit, inital_state)
)

print("")
print("6. Get decoded state:")
add_decode_gate(dynamic_circuit)
show_quantum_state(
    run_dynamic_circuit(dynamic_circuit, inital_state)
)


1. initial state:
(0.196-0.618j) * |000> + (0.558-0.518j) * |001>

2. encoded state:
(0.196-0.618j) * |000> + (0.558-0.518j) * |111>

3. noise applied state:
error_array = [0, 0, 1]
(0.558-0.518j) * |011> + (0.196-0.618j) * |100>

4. parity measurement state:
(0.558-0.518j) * |011> + (0.196-0.618j) * |100>
parity(0, 1): 0
parity(0, 2): 1

5. Apply correction gate:
(0.196-0.618j) * |000> + (0.558-0.518j) * |111>

6. Get decoded state:
(0.196-0.618j) * |000> + (0.558-0.518j) * |001>


Quantum error correction is a framework that allows error correction even when a quantum state has not only bit-flip (Pauli-$X$) but also phase-flip (Pauli-$Z$) errors. Assuming that the errors that occur are probabilistic Pauli operators, the quantum error correction framework is as follows.

- Code: characterized by generative unitary acting on $N$-qubit $U_G$ and a Set of $n-k$ POVMs $\{M_0^{(i)}, M_1^{(i)}\}_i$. They satisfy $\langle \psi,0 | U_G^{\dagger} M_j^{(i)} U_G | \psi,0 \rangle$ for any $k$-qubit state $|\psi\rangle$.
- Before coding: There is a quantum state $|\psi\rangle$ of the computational basis of $k$-qubit.
- Coding: Generate $|\psi' \rangle = U_G (|\psi\rangle \otimes |0\rangle^{n-k})$ with unitary matrix $U_G$.
- Error: after encoding $|\psi' \rangle$, a random Pauli operation $E = X_1^{e_1} X_2 ^{e_2} \ldots X_n^{e_n} Z_1^{e_{n+1}} Z_2 ^{e_{n+2}} \ldots Z_n^{e_{2n}}$ is applied, and gets $E |\psi' \rangle$.
- Error Checking: Measure the state $E |\psi' \rangle$ with $n-k$ POVMs $\{M_0^{(i)}, M_1^{(i)}\}_i$  and obtain $n-k$ bits $s$. Check whether this $s$ is zero or not.
- Decoding: according to the value of $s$, unitary operation $R = X_1^{e'_1} X_2 ^{e'_2} \ldots X_n^{e'_n} Z_1^{e'_{n+1}} Z_2 ^{e'_{n+2}} \ldots Z_n^{e '_{2n}}$ is constructed. The algorithm for finding $R$ from $s$ is called the decoding algorithm of quantum error correction.
- Distance: The minimum weight of $E$ such that $s = 0$ is the distance $d$ of the code. 

There is an essential change in the framework of quantum error correction from classical error correction.
Quantum error correction introduces errors that cause phase inversion. For this reason, the Pauli operator $E$ of the error has a form that includes not only Pauli-$X$ but also Pauli-$Z$.
This change requires that the code be error correctable for phase reversals. The code must also have a distance greater than $1$ for Pauli-$Z$ errors. The classical code rewritten in the framework of a quantum code is a code with $d=1$ for Pauli-$Z$. Also, the syndrome measurement by one code must not destroy the quantum state of $k$-qubit. This corresponds to the fact that any coded state is indistinguishable in $(n-k)$ POVM measurements. Thus, for all POVM elements, $\langle \psi' |M_j^{(i)} | \psi' \rangle$ must be a value independent of $|\psi'\rangle$
 for any $k$-qubit encoded state $|\psi'\rangle = U_B (|\psi\rangle \otimes |0\rangle^{n-k})$.

The stabilizer code described below is a framework that gives $U_B$ and POVM such that the above is satisfied.

Finally, in classical codes, it is possible to obtain $(vG+e)$ by measuring all quantum states in the $Z$ basis for decoding, but in quantum codes, such measurement cannot be done because the $Z$ basis of quantum states destroys the information. Also, the operation used for correction is a Pauli operator that mixes Pauli-$X$ and $Z$. Therefore, the algorithm for estimating the decoding operation $R$ from the obtained syndrome value $s$ alone also needs to be modified for the quantum.

### Stabilizer Codes
Just as the matrix $H_c$ that constitutes parity is given in parity checking codes, there is a framework of stabilizer codes that constitute POVMs in quantum codes.
Instead of directly specifying the code space, stabilizer codes specify the code space using a set of Pauli operators that satisfy certain properties called stabilizer operators.

The generator $S$ of the stabilizer operator is a set $S$ of Pauli operators satisfying the following properties.

- Each element of $S$ is mutually commutative for all. That is, for any $s,s' \in s$, $ss' = s's$.
- Every element of $S$ is independent. That is, for any $s \in S$, the group generated from the set $S\setminus \{s\}$ does not contain $s$.
- The group generated from $S$ does not contain $-I$.

When the above is satisfied, the logical code specified by the stabilizer operator is the subspace composed of quantum states satisfying the following properties.

- $|\psi\rangle$ is an eigenstate with eigenvalue +1 for all $s \in S$.

When there are $l$ independent stabilizer generators, the dimension of the spanned space of such quantum states is $2^{n-l}$.
Thus, if we want to construct a space of $k$ logical qubits in a space of $n$-qubits, we need to have $l = n-k$.

In this code, the syndrome value is given as the result of a projective measurement of the quantum state in $s \in S$.
Since it is the +1 eigenspace of all stabilizer operators that is now used for the calculation, the projective measurement of the stabilizer operator does not destroy the logical qubit of interest.
Now, for simplicity, we assume that one of Pauli$X,Y,Z$ is probabilistically applied to each qubit. Since Pauli$Y$ is the product of $X$ and $Z$, we can regard this as a case where both Pauli$X$ and $Z$ occur.

If we decide to express whether $X$($Z$) error is occurring in the $i$-th qubit in binary $e^{(X)}_i$ ($e^{(Z)}_i$),
measuring a qubit at $Z$ is equivalent to obtaining $e^{(X)}_i$, and the act of measuring at $X$ is equivalent to obtaining $e^{(Z)}_i$.
If we measure with a Pauli operator acting on multiple qubits, we would obtain the XOR of each resulting bit. I.e.,
If $P=Z_2 X_3 Z_4 Z_5$, then the measurement result is $\left(e^{(X)}_2 + e^{(Z)}_3 + e^{(X)}_4 + e^{(X)}_5\right)\bmod 2$.

If $P$ is only a product of Pauli $Z$ and the only error acting on a qubit is a bit flip due to Pauli $X$, then this is synonymous with parity checking, which computes the parity of some bits.
Thus, it can be seen that the stabilizer code is a kind of extension of the parity checking code that allows it to be applied to errors related to Pauli$Z$.

Given a certain stabilizer generator, the distance to its code can be considered as follows.
First, consider the normalization group (Normalizer) for the stabilizer group. A normalization group is a group that satisfies the following equation.

$$
C(S) = \{p \in P \mid \forall s \in S, sp = ps \}
$$

The normalization group minus the stabilizer group, ignoring the difference in global phase, is called the logical operator.
A logic operator is a set of operations that cannot detect errors by the syndrome but change the logical quantum state.
Thus, if an error occurs in enough qubits to execute a logical operator, we cannot detect an error.

Let weight $w(P)$ of $P$ be the number of qubits on which Pauli operator $P$ acts nontrivially (one of Pauli $X,Y,Z$ acts). The code distance is the smallest weight in the set of logical operators, i.e..

$$
d := \min_{P \in C(S)\setminus \langle S \rangle} w(P)
$$

### Topological Codes

Topological codes are codes that originated from the idea of using the degenerate ground state of topological matter as the space of logical qubits. From an experimental point of view, a topological code is a kind of stabilizer code that satisfies the following good properties.

- The parity check matrix $H_c$ is a sparse matrix. Such codes are known to have good performance due to their similarity to low-density parity checking codes, and each syndrome is represented as $O(1)$ parities.
- The data qubits to be checked in each syndrome are spatially clustered within a distance of $O(1)$. Therefore, even in physical systems where CNOT can only be applied to adjacent qubits, it is possible to efficiently measure the syndrome.

Among topological codes, surface codes, with which measurement qubits can also be embedded on a two-dimensional plane, are the closest to realization.

### Computation on stabilizer codes

Since quantum computation introduces errors as the operation is performed, the operation must be performed with the quantum state encoded for the duration of the computation. In this computation, too, there is a significant difference from the classical case.

In parity check codes, the inversion operation on one logical bit corresponds to an inversion operation on at least $O(d)$ and at most $O(n)$ bits. This can obviously be accomplished with $O(n)$ bit flips, which can be completed in 1step if parallelized.

In stabilizer codes, a quantum operation on one logical bit similarly maps to some unitary operation on at least $O(d)$ qubits. Here, the difference from the classical case is that $O(2^d)$ gates are generally required to realize unitary operations on $O(d)$ qubits. Therefore, on a quantum code, even the operation on one logical bit cannot be performed efficiently in general, and the error correction operation will create a new error and the computation on the code will not scale.

On the other hand, depending on the code, there exists $U$ such that the quantum operation $U'$ of $O(d)$ qubits corresponding to the operation $U$ of one logical bit can be completed in $O(1)$ steps. Such an operation on a logic bit that can be efficiently realized on a code is called a transversal operation. If the transversal operation constructs a universal gate set, universal quantum computation can be efficiently performed on the code, but unfortunately, it has been proved that there is no such stabilizer code.

Therefore, it is necessary to somehow procure the missing operations for universal operation. On the surface code, $X, Y, Z, H$, etc. are transversals, and although it is nontrivial whether CNOT is possible, it is known that it can be done efficiently by lattice surgery. Since $S$, $T$, etc. are not transversal, it is possible to create $S$, $T$gate states, etc. with another code and use a method called magic state distillation to gate-teleport these states, thereby creating a universal set.